In [1]:
import tensorflow as tf
import numpy as np
import cv2

# layer, module

In [2]:
class BatchNormalization(tf.keras.layers.BatchNormalization):

    def call(self, x, training=False):
        if not training:
            training = tf.constant(False)
        training = tf.logical_and(training, self.trainable)
        return super().call(x, training)

def convolutional(input_layer,
                  filters_shape,
                  down_sample=False,
                  activate=True,
                  batch_norm=True,
                  regularization=0.0005,
                  reg_stddev=0.01,
                  activate_alpha=0.1):

    if down_sample:
        input_layer = tf.keras.layers.ZeroPadding2D(((1, 0), (1, 0))) (input_layer)
        padding = "valid"
        strides = 2
    else:
        padding = "same"
        strides = 1
    conv = tf.keras.layers.Conv2D(
        filters=filters_shape[-1],
        kernel_size=filters_shape[0],
        strides=strides,
        padding=padding,
        use_bias=not batch_norm,
        kernel_regularizer=tf.keras.regularizers.l2(regularization),
        kernel_initializer=tf.random_normal_initializer(stddev=reg_stddev),
        bias_initializer=tf.constant_initializer(0.))(input_layer)

    if batch_norm:
        conv = BatchNormalization()(conv)
    if activate:
        conv = tf.nn.leaky_relu(conv, alpha=activate_alpha)

    return conv

def res_block(input_layer, input_channel, filter_num1, filter_num2):
	short_cut = input_layer
	conv = convolutional(input_layer, filters_shape=(1,1,input_layer,filter_num1))
	conv = convolutional(conv, filters_shape=(3,3,filter_num1,filter_num2))

	res_output = short_cut+ conv 
	return res_output

def upsample(input_layer):
    return tf.image.resize(
        input_layer, (input_layer.shape[1] * 2, input_layer.shape[2] * 2),
        method='nearest')

# darknet 53

In [3]:
def darknet53(input_data):
    input_data = convolutional(input_data, (3, 3, 3, 32))
    input_data = convolutional(input_data, (3, 3, 32, 64), down_sample=True)

    for i in range(1):
        input_data = res_block(input_data, 64, 32, 64)

    input_data = convolutional(input_data, (3, 3, 64, 128), down_sample=True)

    for i in range(2):
        input_data = res_block(input_data, 128, 64, 128)

    input_data = convolutional(input_data, (3, 3, 128, 256), down_sample=True)

    for i in range(8):
        input_data = res_block(input_data, 256, 128, 256)

    route_1 = input_data

    input_data = convolutional(input_data, (3, 3, 256, 512), down_sample=True)

    for i in range(8):
        input_data = res_block(input_data, 512, 256, 512)
    route_2 = input_data
    input_data = convolutional(input_data, (3, 3, 512, 1024), down_sample=True)

    for i in range(4):
        input_data = res_block(input_data, 1024, 512, 1024)

    return route_1, route_2, input_data

# yolov3

In [4]:
# hyperparameters 
NUM_CLASSES = 2
STRIDES = np.array([8,16,32])
ANCHORS =(1.25,1.625, 2.0,3.75, 4.125,2.875, 1.875,3.8125, 3.875,2.8125, 3.6875,7.4375, 3.625,2.8125, 4.875,6.1875, 11.65625,10.1875)
ANCHORS = np.array(ANCHORS).reshape(3,3,2)

In [5]:
# 주석해제 했을때 모델 크기
# Total params: 61,581,727
# Trainable params: 61,529,119
# Non-trainable params: 52,608

# 주석처리 했을때 모델 크기
# Total params: 47,797,663
# Trainable params: 47,755,807
# Non-trainable params: 41,856


def yoloV3(input_layer):
    route_1, route_2, conv = darknet53(input_layer)

    conv = convolutional(conv, (1, 1, 1024, 512))
#     conv = convolutional(conv, (3, 3, 512, 1024))
#     conv = convolutional(conv, (1, 1, 1024, 512))
#     conv = convolutional(conv, (3, 3, 512, 1024))
#     conv = convolutional(conv, (1, 1, 1024, 512))

    conv_lobj_branch = convolutional(conv, (3, 3, 512, 1024))
    conv_lbbox = convolutional(conv_lobj_branch,
                               (1, 1, 1024, 3 * (NUM_CLASSES + 5)),
                               activate=False,
                               batch_norm=False)

    conv = convolutional(conv, (1, 1, 512, 256))
    conv = upsample(conv)

    conv = tf.concat([conv, route_2], axis=-1)
    conv = convolutional(conv, (1, 1, 768, 256))
#     conv = convolutional(conv, (3, 3, 256, 512))
#     conv = convolutional(conv, (1, 1, 512, 256))
#     conv = convolutional(conv, (3, 3, 256, 512))
#     conv = convolutional(conv, (1, 1, 512, 256))

    conv_mobj_branch = convolutional(conv, (3, 3, 256, 512))
    conv_mbbox = convolutional(conv_mobj_branch,
                               (1, 1, 512, 3 * (NUM_CLASSES + 5)),
                               activate=False,
                               batch_norm=False)

    conv = convolutional(conv, (1, 1, 256, 128))
    conv = upsample(conv)

    conv = tf.concat([conv, route_1], axis=-1)

    conv = convolutional(conv, (1, 1, 384, 128))
#     conv = convolutional(conv, (3, 3, 128, 256))
#     conv = convolutional(conv, (1, 1, 256, 128))
#     conv = convolutional(conv, (3, 3, 128, 256))
#     conv = convolutional(conv, (1, 1, 256, 128))

    conv_sobj_branch = convolutional(conv, (3, 3, 128, 256))
    conv_sbbox = convolutional(conv_sobj_branch,
                               (1, 1, 256, 3 * (NUM_CLASSES + 5)),
                               activate=False,
                               batch_norm=False)
    return [conv_sbbox, conv_mbbox, conv_lbbox]

# decoder

In [6]:
def decode(conv_out, i = 0):
	conv_shape = tf.shape(conv_out)
	batch_size = conv_shape[0]
	output_size = conv_shape[1]

	conv_output = tf.reshape(conv_out, (batch_size, output_size,output_size, 3,5+NUM_CLASSES))
	
	conv_raw_dxdy = conv_output[:,:,:,:,0:2]
	conv_raw_dwdh = conv_output[:,:,:,:,2:4]
	conv_raw_conf = conv_output[:,:,:,:,4:5]
	conv_raw_prob = conv_output[:,:,:,:,5:]

	y = tf.tile(tf.range(output_size,dtype=tf.int32)[:,tf.newaxis],[1,output_size])
	x = tf.tile(tf.range(output_size, dtype= tf.int32)[tf.newaxis,:],[output_size,1])

	xy_grid = tf.concat([x[:,:,tf.newaxis],y[:,:,tf.newaxis]], axis = -1)
	xy_grid = tf.tile(xy_grid[tf.newaxis,:,:,tf.newaxis,:],[batch_size,1,1,3,1])
	xy_grid = tf.cast(xy_grid,tf.float32)

	pred_xy = (tf.sigmoid(conv_raw_dxdy)+xy_grid)*STRIDES[i]
	pred_wh = (tf.exp(conv_raw_dwdh)*ANCHORS[i])*STRIDES[i]
	pred_xywh = tf.concat([pred_xy,pred_wh], axis = -1)

	pred_conf = tf.sigmoid(conv_raw_conf)
	pred_prob = tf.sigmoid(conv_raw_prob)

	return tf.concat([pred_xywh, pred_conf, pred_prob], axis = -1)

In [7]:
def Model():
    input_layer = tf.keras.layers.Input([416, 416, 3])
    feature_maps = yoloV3(input_layer)

    bbox_tensors = []

    for i, fm in enumerate(feature_maps):
        bbox_tensor = decode(fm, i)
        bbox_tensors.append(bbox_tensor)

    model = tf.keras.Model(input_layer, bbox_tensors)
    # 	model = Load_weights(model, weight_file)

    return model

In [8]:
model = Model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 416, 416, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 416, 416, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 416, 416, 32  128        ['conv2d[0][0]']                 
 alization)                     )                                                             

                                                                                                  
 batch_normalization_7 (BatchNo  (None, 104, 104, 64  256        ['conv2d_7[0][0]']               
 rmalization)                   )                                                                 
                                                                                                  
 tf.nn.leaky_relu_7 (TFOpLambda  (None, 104, 104, 64  0          ['batch_normalization_7[0][0]']  
 )                              )                                                                 
                                                                                                  
 conv2d_8 (Conv2D)              (None, 104, 104, 12  73728       ['tf.nn.leaky_relu_7[0][0]']     
                                8)                                                                
                                                                                                  
 batch_nor

                                                                                                  
 tf.__operators__.add_5 (TFOpLa  (None, 52, 52, 256)  0          ['tf.__operators__.add_4[0][0]', 
 mbda)                                                            'tf.nn.leaky_relu_15[0][0]']    
                                                                                                  
 conv2d_16 (Conv2D)             (None, 52, 52, 128)  32768       ['tf.__operators__.add_5[0][0]'] 
                                                                                                  
 batch_normalization_16 (BatchN  (None, 52, 52, 128)  512        ['conv2d_16[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 tf.nn.leaky_relu_16 (TFOpLambd  (None, 52, 52, 128)  0          ['batch_normalization_16[0][0]'] 
 a)       

 ormalization)                                                                                    
                                                                                                  
 tf.nn.leaky_relu_24 (TFOpLambd  (None, 52, 52, 128)  0          ['batch_normalization_24[0][0]'] 
 a)                                                                                               
                                                                                                  
 conv2d_25 (Conv2D)             (None, 52, 52, 256)  294912      ['tf.nn.leaky_relu_24[0][0]']    
                                                                                                  
 batch_normalization_25 (BatchN  (None, 52, 52, 256)  1024       ['conv2d_25[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 tf.nn.lea

                                                                                                  
 conv2d_33 (Conv2D)             (None, 26, 26, 256)  131072      ['tf.__operators__.add_13[0][0]']
                                                                                                  
 batch_normalization_33 (BatchN  (None, 26, 26, 256)  1024       ['conv2d_33[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 tf.nn.leaky_relu_33 (TFOpLambd  (None, 26, 26, 256)  0          ['batch_normalization_33[0][0]'] 
 a)                                                                                               
                                                                                                  
 conv2d_34 (Conv2D)             (None, 26, 26, 512)  1179648     ['tf.nn.leaky_relu_33[0][0]']    
          

...

 ormalization)                                                                                    
                                                                                                  
 batch_normalization_53 (BatchN  (None, 13, 13, 1024  4096       ['conv2d_53[0][0]']              
 ormalization)                  )                                                                 
                                                                                                  
 tf.nn.leaky_relu_59 (TFOpLambd  (None, 52, 52, 256)  0          ['batch_normalization_59[0][0]'] 
 a)                                                                                               
                                                                                                  
 tf.nn.leaky_relu_56 (TFOpLambd  (None, 26, 26, 512)  0          ['batch_normalization_56[0][0]'] 
 a)                                                                                               
          

                                                                                                  
 tf.tile_4 (TFOpLambda)         (26, 26)             0           ['tf.__operators__.getitem_18[0][
                                                                 0]',                             
                                                                  'tf.__operators__.getitem_12[0][
                                                                 0]']                             
                                                                                                  
 tf.tile_3 (TFOpLambda)         (26, 26)             0           ['tf.__operators__.getitem_17[0][
                                                                 0]',                             
                                                                  'tf.__operators__.getitem_12[0][
                                                                 0]']                             
          

                                                                                                  
 tf.reshape_2 (TFOpLambda)      (None, 13, 13, 3, 7  0           ['conv2d_54[0][0]',              
                                )                                 'tf.__operators__.getitem_22[0][
                                                                 0]',                             
                                                                  'tf.__operators__.getitem_23[0][
                                                                 0]',                             
                                                                  'tf.__operators__.getitem_23[0][
                                                                 0]']                             
                                                                                                  
 tf.__operators__.getitem_32 (S  (1, 13, 13, 1, 2)   0           ['tf.concat_8[0][0]']            
 licingOpL

                                                                                                  
 tf.__operators__.add_25 (TFOpL  (None, 13, 13, 3, 2  0          ['tf.math.sigmoid_6[0][0]',      
 ambda)                         )                                 'tf.cast_2[0][0]']              
                                                                                                  
 tf.math.multiply_7 (TFOpLambda  (None, 13, 13, 3, 2  0          ['tf.math.exp_2[0][0]']          
 )                              )                                                                 
                                                                                                  
 tf.math.multiply (TFOpLambda)  (None, 52, 52, 3, 2  0           ['tf.__operators__.add_23[0][0]']
                                )                                                                 
                                                                                                  
 tf.math.m

Total params: 47,797,663
Trainable params: 47,755,807
Non-trainable params: 41,856
__________________________________________________________________________________________________
